In [2]:
import pandas as pd
names = pd.read_csv('~/wasm-type-prediction/data/extracted/name-stats/name-stats.csv')

In [3]:
# Most common name, by number of binaries it appears in.
names.name.value_counts()[:20]

size_t                                                               12766
basic_string<char, std::char_traits<char>, std::allocator<char> >     4309
_IO_FILE                                                              4231
FILE                                                                  3986
uint32_t                                                              3754
string                                                                3457
size_type                                                             3351
length_error                                                          3085
basic_ostream<char, std::char_traits<char> >                          3064
ios_base                                                              3012
uint8_t                                                               2974
ostreambuf_iterator<char, std::char_traits<char> >                    2928
type                                                                  2338
ptrdiff_t                

In [25]:
# Most common name, by overall count
name_counts = names.groupby('name')['count'].sum().sort_values(ascending=False)
name_counts

name
size_type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          686352
size_t                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [5]:
commonpath = os.path.commonpath(list(names.file.unique()))
print(f"common directory of all samples: '{commonpath}'")

def first_dir(path):
    path_relative = os.path.relpath(path, commonpath)
    first_dir = path_relative.split(os.sep)[0]
    return first_dir

names['project'] = names.apply(lambda x: first_dir(x['file']), axis=1)
names

common directory of all samples: 'binaries/wasm-dwarf'


,name,file,count,project
0,"__tuple_leaf<1, std::shared_ptr<fst::DefaultCo...",binaries/wasm-dwarf/libfst-dev/src/openfst-1.6...,2,libfst-dev
1,"__tuple_leaf<1, std::shared_ptr<fst::DefaultCo...",binaries/wasm-dwarf/libfst-dev/src/openfst-1.6...,2,libfst-dev
2,"__tuple_leaf<1, std::shared_ptr<fst::DefaultCo...",binaries/wasm-dwarf/libfst-dev/src/openfst-1.6...,2,libfst-dev
3,"__tuple_leaf<1, std::shared_ptr<fst::DefaultCo...",binaries/wasm-dwarf/libfst-dev/src/openfst-1.6...,2,libfst-dev
4,"__tuple_leaf<1, std::shared_ptr<fst::DefaultCo...",binaries/wasm-dwarf/libfst-dev/src/openfst-1.6...,2,libfst-dev
...,...,...,...,...
1317022,MidiPlayer,binaries/wasm-dwarf/scummvm/src/scummvm-2.0.0+...,1,scummvm
1317023,MidiPlayer,binaries/wasm-dwarf/scummvm/src/scummvm-2.0.0+...,1,scummvm
1317024,MidiPlayer,binaries/wasm-dwarf/scummvm/src/scummvm-2.0.0+...,1,scummvm
1317025,MidiPlayer,binaries/wasm-dwarf/scummvm/src/scummvm-2.0.0+...,1,scummvm


In [6]:
print('unique projects:', len(names.project.unique()))

from collections import Counter
print('project names, by number of unique names in them:')
Counter(names.project).most_common(50)

unique projects: 1346
project names, by number of unique names in them:


[('libfst-dev', 710580),
 ('scummvm', 48049),
 ('cmake', 42361),
 ('libxerces-c-dev', 35610),
 ('gdal-bin', 18831),
 ('leptonica-progs', 14217),
 ('cppcheck', 12999),
 ('liblog4cplus-1.1-9', 10240),
 ('emboss', 8127),
 ('gringo', 7250),
 ('libopenscenegraph-3.4-131', 7024),
 ('nlohmann-json-dev', 6989),
 ('libncl-dev', 6930),
 ('icu-devtools', 6780),
 ('libopenscenegraph-dev', 6010),
 ('libgloox-dbg', 5991),
 ('libamd2', 5708),
 ('hippomocks', 5679),
 ('imagemagick', 5650),
 ('libcrypto++-dev', 5644),
 ('kwstyle', 5531),
 ('libopengm-bin', 5164),
 ('libitpp-dev', 4934),
 ('aria2', 4675),
 ('libprotozero-dev', 4565),
 ('tripwire', 4390),
 ('libspdlog-dev', 4304),
 ('capnproto', 4203),
 ('flatzinc', 4171),
 ('residualvm', 3743),
 ('libmbedcrypto1', 3618),
 ('gsl-bin', 3615),
 ('libaria-demo', 3565),
 ('kimwitu++', 3421),
 ('libyaz5', 3408),
 ('libbpp-core-dev', 3273),
 ('libyaml-cpp0.3-dev', 3118),
 ('leveldb-doc', 3077),
 ('libmyth', 2836),
 ('voronota', 2826),
 ('libsegyio-dev', 2802),

In [9]:
# Most common names, by appearances in projects
names_projects = names.groupby('name')['project'].nunique().sort_values(ascending=False)
names_projects

name
size_t                                                                                                                          859
_IO_FILE                                                                                                                        636
FILE                                                                                                                            608
uint32_t                                                                                                                        333
basic_string<char, std::char_traits<char>, std::allocator<char> >                                                               232
                                                                                                                               ... 
__tuple_leaf<0, const fst::Fst<fst::GallicArc<fst::ArcTpl<fst::TropicalWeightTpl<float> >, fst::GALLIC_MIN> > &, false>           1
__tuple_leaf<0, const fst::Fst<fst::GallicArc<fst::ArcTpl<fst::Tropical

In [27]:
names_proportion = names_projects / len(names.project.unique())
names_proportion = names_proportion.to_frame().rename({'project': 'project proportion'}, axis=1)
names_proportion = names_proportion.join(name_counts)
print(names_proportion)
names_proportion.to_csv('~/wasm-type-prediction/data/extracted/name-stats/name-stats.project-proportion.csv')

                                                    project proportion   count
name                                                                          
size_t                                                        0.638187  516451
_IO_FILE                                                      0.472511   22493
FILE                                                          0.451709   20949
uint32_t                                                      0.247400   54395
basic_string<char, std::char_traits<char>, std:...            0.172363  135900
...                                                                ...     ...
__tuple_leaf<0, const fst::Fst<fst::GallicArc<f...            0.000743      80
__tuple_leaf<0, const fst::Fst<fst::GallicArc<f...            0.000743      80
__tuple_leaf<0, const fst::Fst<fst::GallicArc<f...            0.000743      78
__tuple_leaf<0, const fst::Fst<fst::ReverseArc<...            0.000743      88
A                                                   

In [47]:
import re

INT_TYPE_RE = re.compile('u?_?int(\d+)?(_t)?')
# size_type as well, since it's the C++ equivalent of size_t and always a typedef to size_t anyway.
OTHER_PRIMITIVE_TYPE = ['int_type', 'char_type', 'value_type', 'type', 'bool', 'pointer', 'size_type']

PROJECT_THRESHOLD = 0.01  # 1% of all projects should have the name

names_to_keep = []
# Keep only names that are NOT project-specific and "commonly used".
for name, series in names_proportion[names_proportion['project proportion'] > PROJECT_THRESHOLD].iterrows():
    # If it matches a primitive type, don't keep the name since we can just use the primitive name.
    # If name starts with underscore, it is an internal name, we don't want to keep.
    if not INT_TYPE_RE.match(name.lower()) and name.lower() not in OTHER_PRIMITIVE_TYPE and not name.startswith('_'):
        # print(name, prop)
        names_to_keep.append(name)

print(len(names_to_keep))

with open('/home/daniel/wasm-type-prediction/data/extracted/name-stats/names-keep.txt', 'w') as f:
    for line in names_to_keep:
        f.write(f"{line}\n")

239
                                                    project proportion   count
name                                                                          
size_t                                                        0.638187  516451
FILE                                                          0.451709   20949
basic_string<char, std::char_traits<char>, std:...            0.172363  135900
basic_ostream<char, std::char_traits<char> >                  0.162704   35460
ios_base                                                      0.161218   10002
...                                                                ...     ...
sockaddr_storage                                              0.010401     175
code                                                          0.010401      48
gz_header                                                     0.010401      41
libfdata_mapped_range_t                                       0.010401      90
gz_header_s                                     

In [59]:
names_proportion = names_proportion[names_proportion.index.isin(names_to_keep)]
print(names_proportion)
names_proportion.to_csv('~/wasm-type-prediction/data/extracted/name-stats/name-stats.project-proportion.keep.csv')

for name, series in list(names_proportion.iterrows())[:10]:
    escaped_name = name.replace("_", r"\_")
    print(f'\\code{{{escaped_name}}} & {series["count"]:,.0f} & {series["project proportion"] * 100:2.1f}\\,\\% \\\\')

                                                    project proportion   count
name                                                                          
size_t                                                        0.638187  516451
allocator_type                                                0.064636  157511
basic_string<char, std::char_traits<char>, std:...            0.172363  135900
const_pointer                                                 0.063150   45222
string                                                        0.155275   45081
...                                                                ...     ...
gz_header_s                                                   0.010401      41
libcthreads_internal_thread_pool                              0.014859      25
libcthreads_internal_thread_pool_t                            0.014859      25
libfdata_internal_list_t                                      0.011144      22
libfdata_internal_list                              